# Removing redundant features

In [1]:
import pandas as pd
volunteer = pd.read_csv('datasets/volunteer_opportunities.csv')
volunteer.head()

,opportunity_id,content_id,vol_requests,event_time,title,hits,summary,is_priority,category_id,category_desc,amsl,amsl_unit,org_title,org_content_id,addresses_count,locality,region,postalcode,primary_loc,display_url,recurrence_type,hours,created_date,last_modified_date,start_date_date,end_date_date,status,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,4996,37004,50,0,Volunteers Needed For Rise Up & Stay Put! Home...,737,Building on successful events last summer and ...,NaN,NaN,NaN,NaN,NaN,Center For NYC Neighborhoods,4426,1,NaN,NY,NaN,NaN,/opportunities/4996,onetime,0,January 13 2011,June 23 2011,July 30 2011,July 30 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5008,37036,2,0,Web designer,22,Build a website for an Afghan business,NaN,1.0,Strengthening Communities,NaN,NaN,Bpeace,37026,1,"5 22nd St\nNew York, NY 10010\n(40.74053152272...",NY,10010.0,NaN,/opportunities/5008,onetime,0,January 14 2011,January 25 2011,February 01 2011,February 01 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5016,37143,20,0,Urban Adventures - Ice Skating at Lasker Rink,62,Please join us and the students from Mott Hall...,NaN,1.0,Strengthening Communities,NaN,NaN,Street Project,3001,1,NaN,NY,10026.0,NaN,/opportunities/5016,onetime,0,January 19 2011,January 21 2011,January 29 2011,January 29 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5022,37237,500,0,Fight global hunger and support women farmers ...,14,The Oxfam Action Corps is a group of dedicated...,NaN,1.0,Strengthening Communities,NaN,NaN,Oxfam America,2170,1,NaN,NY,2114.0,NaN,/opportunities/5022,ongoing,0,January 21 2011,January 25 2011,February 14 2011,March 31 2012,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5055,37425,15,0,Stop 'N' Swap,31,Stop 'N' Swap reduces NYC's waste by finding n...,NaN,4.0,Environment,NaN,NaN,Office of Recycling Outreach and Education,36773,1,NaN,NY,10455.0,NaN,/opportunities/5055,onetime,0,January 28 2011,February 01 2011,February 05 2011,February 05 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Dropping redundant features similar to "location" remaining only "postalcode"
volunteer.drop(['locality','region','primary_loc','Latitude','Longitude'],axis=1,inplace=True)
volunteer.head()

,opportunity_id,content_id,vol_requests,event_time,title,hits,summary,is_priority,category_id,category_desc,amsl,amsl_unit,org_title,org_content_id,addresses_count,postalcode,display_url,recurrence_type,hours,created_date,last_modified_date,start_date_date,end_date_date,status,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,4996,37004,50,0,Volunteers Needed For Rise Up & Stay Put! Home...,737,Building on successful events last summer and ...,NaN,NaN,NaN,NaN,NaN,Center For NYC Neighborhoods,4426,1,NaN,/opportunities/4996,onetime,0,January 13 2011,June 23 2011,July 30 2011,July 30 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN
1,5008,37036,2,0,Web designer,22,Build a website for an Afghan business,NaN,1.0,Strengthening Communities,NaN,NaN,Bpeace,37026,1,10010.0,/opportunities/5008,onetime,0,January 14 2011,January 25 2011,February 01 2011,February 01 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN
2,5016,37143,20,0,Urban Adventures - Ice Skating at Lasker Rink,62,Please join us and the students from Mott Hall...,NaN,1.0,Strengthening Communities,NaN,NaN,Street Project,3001,1,10026.0,/opportunities/5016,onetime,0,January 19 2011,January 21 2011,January 29 2011,January 29 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN
3,5022,37237,500,0,Fight global hunger and support women farmers ...,14,The Oxfam Action Corps is a group of dedicated...,NaN,1.0,Strengthening Communities,NaN,NaN,Oxfam America,2170,1,2114.0,/opportunities/5022,ongoing,0,January 21 2011,January 25 2011,February 14 2011,March 31 2012,approved,NaN,NaN,NaN,NaN,NaN,NaN
4,5055,37425,15,0,Stop 'N' Swap,31,Stop 'N' Swap reduces NYC's waste by finding n...,NaN,4.0,Environment,NaN,NaN,Office of Recycling Outreach and Education,36773,1,10455.0,/opportunities/5055,onetime,0,January 28 2011,February 01 2011,February 05 2011,February 05 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN


# Removing highly correlated features

In [3]:
wine = pd.read_csv('datasets/wine_types.csv')

In [4]:
# Print out the column correlations of the wine dataset
wine.corr()

,Type,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
Type,1.000000,-0.328222,0.437776,-0.049643,0.517859,-0.209179,-0.719163,-0.847498,0.489109,-0.499130,0.265668,-0.617369,-0.788230,-0.633717
Alcohol,-0.328222,1.000000,0.094397,0.211545,-0.310235,0.270798,0.289101,0.236815,-0.155929,0.136698,0.546364,-0.071747,0.072343,0.643720
Malic acid,0.437776,0.094397,1.000000,0.164045,0.288500,-0.054575,-0.335167,-0.411007,0.292977,-0.220746,0.248985,-0.561296,-0.368710,-0.192011
Ash,-0.049643,0.211545,0.164045,1.000000,0.443367,0.286587,0.128980,0.115077,0.186230,0.009652,0.258887,-0.074667,0.003911,0.223626
Alcalinity of ash,0.517859,-0.310235,0.288500,0.443367,1.000000,-0.083333,-0.321113,-0.351370,0.361922,-0.197327,0.018732,-0.273955,-0.276769,-0.440597
Magnesium,-0.209179,0.270798,-0.054575,0.286587,-0.083333,1.000000,0.214401,0.195784,-0.256294,0.236441,0.199950,0.055398,0.066004,0.393351
Total phenols,-0.719163,0.289101,-0.335167,0.128980,-0.321113,0.214401,1.000000,0.864564,-0.449935,0.612413,-0.055136,0.433681,0.699949,0.498115
Flavanoids,-0.847498,0.236815,-0.411007,0.115077,-0.351370,0.195784,0.864564,1.000000,-0.537900,0.652692,-0.172379,0.543479,0.787194,0.494193
Nonflavanoid phenols,0.489109,-0.155929,0.292977,0.186230,0.361922,-0.256294,-0.449935,-0.537900,1.000000,-0.365845,0.139057,-0.262640,-0.503270,-0.311385
Proanthocyanins,-0.499130,0.136698,-0.220746,0.009652,-0.197327,0.236441,0.612413,0.652692,-0.365845,1.000000,-0.025250,0.295544,0.519067,0.330417


In [5]:
# Drop that column from the DataFrame that have a correlation coefficient above 0.75 with at least two other columns.
wine = wine.drop(['Flavanoids'], axis=1)

wine.head()

,Type,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,0.39,1.82,4.32,1.04,2.93,735


# Selecting features using text vectors

In [6]:
volunteer.dropna(subset=['category_desc'],inplace=True)

In [7]:
text_data = volunteer['title']
text_data

1                                           Web designer
2          Urban Adventures - Ice Skating at Lasker Rink
3      Fight global hunger and support women farmers ...
4                                          Stop 'N' Swap
5                                   Queens Stop 'N' Swap
                             ...                        
660            Volunteer for NYLAG's Food Stamps Project
661      Iridescent Science Studio Open House Volunteers
662                                    French Translator
663                    Marketing & Advertising Volunteer
664    Volunteer filmmakers to help Mayor's Office wi...
Name: title, Length: 617, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer
tfidf_vec = TfidfVectorizer()

# Fit on the text_data
text_tfidf = tfidf_vec.fit_transform(text_data)
text_tfidf

<617x1089 sparse matrix of type '<class 'numpy.float64'>'
	with 3172 stored elements in Compressed Sparse Row format>

### Original vocab : vocabulary in the text_data along its unique identifier

In [9]:
# Vocabulary list
print(tfidf_vec.vocabulary_)

{'web': 1048, 'designer': 278, 'urban': 1017, 'adventures': 38, 'ice': 490, 'skating': 890, 'at': 90, 'lasker': 559, 'rink': 832, 'fight': 368, 'global': 423, 'hunger': 487, 'and': 68, 'support': 944, 'women': 1061, 'farmers': 356, 'join': 535, 'the': 969, 'oxfam': 708, 'action': 27, 'corps': 240, 'in': 498, 'nyc': 680, 'stop': 922, 'swap': 947, 'queens': 790, 'staff': 911, 'development': 281, 'trainer': 992, 'claro': 200, 'brooklyn': 145, 'volunteer': 1037, 'attorney': 93, 'community': 221, 'health': 455, 'advocates': 43, 'supervise': 942, 'children': 189, 'highland': 466, 'park': 717, 'garden': 409, 'worldofmoney': 1071, 'org': 696, 'youth': 1085, 'amazing': 60, 'race': 791, 'qualified': 789, 'board': 133, 'member': 620, 'seats': 860, 'available': 98, 'young': 1083, 'adult': 33, 'tutor': 1006, 'updated': 1016, '30': 11, '11': 0, 'insurance': 513, 'claims': 199, 'manager': 600, 'timebanksnyc': 979, 'great': 432, 'exchange': 340, 'clean': 205, 'up': 1015, 'asbury': 81, 'cementary': 171

### Exploring row data

**Note:** Row data contains two components we'll need: the word weights and the index of the word


In [10]:
# word weight
print(text_tfidf[5].data) #<-- gives weight of each word in fourth row

[0.62268351 0.47384649 0.62268351]


In [11]:
# index of the word
print(text_tfidf[5].indices) #<-- gives index of each word in fourth row

[992 281 911]


### Reversed vocab : unique identifier representing vocabulary in text_data

Before putting together the vocabulary, the word indices, and their weights, we want to reverse the key value pairs in the vocabulary. It'll be easier later on if we have the index number in the key position in the dictionary.

In [12]:
# Reverse the vocabulary dictionary 
vocab = {v:k for k,v in tfidf_vec.vocabulary_.items()}
print(vocab)

{1048: 'web', 278: 'designer', 1017: 'urban', 38: 'adventures', 490: 'ice', 890: 'skating', 90: 'at', 559: 'lasker', 832: 'rink', 368: 'fight', 423: 'global', 487: 'hunger', 68: 'and', 944: 'support', 1061: 'women', 356: 'farmers', 535: 'join', 969: 'the', 708: 'oxfam', 27: 'action', 240: 'corps', 498: 'in', 680: 'nyc', 922: 'stop', 947: 'swap', 790: 'queens', 911: 'staff', 281: 'development', 992: 'trainer', 200: 'claro', 145: 'brooklyn', 1037: 'volunteer', 93: 'attorney', 221: 'community', 455: 'health', 43: 'advocates', 942: 'supervise', 189: 'children', 466: 'highland', 717: 'park', 409: 'garden', 1071: 'worldofmoney', 696: 'org', 1085: 'youth', 60: 'amazing', 791: 'race', 789: 'qualified', 133: 'board', 620: 'member', 860: 'seats', 98: 'available', 1083: 'young', 33: 'adult', 1006: 'tutor', 1016: 'updated', 11: '30', 0: '11', 513: 'insurance', 199: 'claims', 600: 'manager', 979: 'timebanksnyc', 432: 'great', 340: 'exchange', 205: 'clean', 1015: 'up', 81: 'asbury', 171: 'cementary'

### Zipping row data outputs : vocabulary indices and its weigth

In [28]:
# zip together the row indices and weights
zipped_row = dict(zip(text_tfidf[563].indices,
                     text_tfidf[563].data))
zipped_row

{963: 0.4198213393570348,
 101: 0.4198213393570348,
 690: 0.28638900522683663,
 14: 0.28638900522683663,
 172: 0.28282519925488286,
 459: 0.24695145752725162,
 353: 0.32604266971232865,
 380: 0.1614984208472933,
 161: 0.32604266971232865,
 969: 0.2285293730838633,
 90: 0.2270959851215777}

### Mapping vocabulary to its weight for given row 

In [29]:
zipped = dict(zip(text_tfidf[563].indices,
                  text_tfidf[563].data))
zipped

{963: 0.4198213393570348,
 101: 0.4198213393570348,
 690: 0.28638900522683663,
 14: 0.28638900522683663,
 172: 0.28282519925488286,
 459: 0.24695145752725162,
 353: 0.32604266971232865,
 380: 0.1614984208472933,
 161: 0.32604266971232865,
 969: 0.2285293730838633,
 90: 0.2270959851215777}

In [30]:
dict_of_words={}

for i in text_tfidf[563].indices:
    dict_of_words[vocab[i]] = zipped[i]
    
dict_of_words

{'teen': 0.4198213393570348,
 'babies': 0.4198213393570348,
 'only': 0.28638900522683663,
 '55': 0.28638900522683663,
 'center': 0.28282519925488286,
 'help': 0.24695145752725162,
 'family': 0.32604266971232865,
 'for': 0.1614984208472933,
 'care': 0.32604266971232865,
 'the': 0.2285293730838633,
 'at': 0.2270959851215777}

### Mapping vocabulary to its weight for given row using function

In [31]:
# mapping of words to its scores
def return_weights(vocab, vector, vector_index):
    
    zipped = dict(zip(vector[vector_index].indices,
                    vector[vector_index].data))
    
    return {vocab[i]:zipped[i] for i in vector[vector_index].indices}

print(return_weights(vocab, text_tfidf, 563))

{'teen': 0.4198213393570348, 'babies': 0.4198213393570348, 'only': 0.28638900522683663, '55': 0.28638900522683663, 'center': 0.28282519925488286, 'help': 0.24695145752725162, 'family': 0.32604266971232865, 'for': 0.1614984208472933, 'care': 0.32604266971232865, 'the': 0.2285293730838633, 'at': 0.2270959851215777}


### Extracting text vectors: Part 1

In [32]:
# Add in the rest of the arguments
def return_weights(vocab, original_vocab, vector, vector_index, top_n):
    zipped = dict(zip(vector[vector_index].indices, vector[vector_index].data))
    
    # Transform that zipped dict into a series
    zipped_series = pd.Series({vocab[i]:zipped[i] for i in vector[vector_index].indices})
    
    # Sort the series to pull out the top n weighted words
    zipped_index = zipped_series.sort_values(ascending=False)[:top_n].index
    return [original_vocab[i] for i in zipped_index]

# Print out the weighted words
print(return_weights(vocab, tfidf_vec.vocabulary_, text_tfidf, 563, 3))

[963, 101, 353]


#### Undertanding the function

In [34]:
# Dict --> word's unique identifier : word's weight
zipped = dict(zip(text_tfidf[563].indices, text_tfidf[563].data))
zipped

{963: 0.4198213393570348,
 101: 0.4198213393570348,
 690: 0.28638900522683663,
 14: 0.28638900522683663,
 172: 0.28282519925488286,
 459: 0.24695145752725162,
 353: 0.32604266971232865,
 380: 0.1614984208472933,
 161: 0.32604266971232865,
 969: 0.2285293730838633,
 90: 0.2270959851215777}

In [35]:
# Series of zipped, instead, the word's unique identifier is replaced with actual word
zipped_series = pd.Series({vocab[i]:zipped[i] for i in text_tfidf[563].indices})
zipped_series

teen      0.419821
babies    0.419821
only      0.286389
55        0.286389
center    0.282825
help      0.246951
family    0.326043
for       0.161498
care      0.326043
the       0.228529
at        0.227096
dtype: float64

In [36]:
# Sort the series to pull out the top n weighted words
zipped_index = zipped_series.sort_values(ascending=False)[:3].index
zipped_index

Index(['teen', 'babies', 'family'], dtype='object')

In [37]:
# List of unique identifier for given top n weighted words
[tfidf_vec.vocabulary_[i] for i in zipped_index]

[963, 101, 353]

### Extracting text vectors: Part 2

In [38]:
def words_to_filter(vocab, original_vocab, vector, top_n):
    filter_list = []

    # for all documents
    for i in range(0, vector.shape[0]):
    
        # Call the return_weights function and extend filter_list
        filtered = return_weights(vocab, original_vocab, vector, i, top_n)
        filter_list.extend(filtered)
        
    # Return the list in a set, so we don't get duplicate word indices
    return set(filter_list)

In [39]:
# Call the function to get the list of word indices
filtered_words = words_to_filter(vocab, tfidf_vec.vocabulary_, text_tfidf, 3)
print(filtered_words)

{1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 15, 16, 18, 19, 20, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 65, 66, 67, 69, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 

In [40]:
# Filter the columns in text_tfidf to only those in filtered_words
filtered_text = text_tfidf[:, list(filtered_words)]
filtered_text

<617x1007 sparse matrix of type '<class 'numpy.float64'>'
	with 2946 stored elements in Compressed Sparse Row format>

In [41]:
text_tfidf.shape

(617, 1089)

The feature was reduced from 1089 to 1007.

# Training Naive Bayes with feature selection

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

y = volunteer['category_desc']

# Split the dataset according to the class distribution of category_desc
X_train, X_test, y_train, y_test = train_test_split(filtered_text.toarray(), y, stratify=y, random_state=42)

# Fit the model to the training data
nb.fit(X_train,y_train)

# Print out the model's accuracy
print(nb.score(X_test,y_test))

0.5161290322580645


# Dimension Reductionality

In [43]:
from sklearn.decomposition import PCA

# Instantiate a PCA object
pca = PCA()

# Define the features and labels from the wine dataset
X = wine.drop('Type', axis=1)
y = wine["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Apply PCA to the wine dataset X vector
pca_X_train = pca.fit_transform(X_train)
pca_X_test = pca.transform(X_test)

# Look at the percentage of variance explained by the different components
print(pca.explained_variance_ratio_)

[9.97802349e-01 2.02071713e-03 9.82348559e-05 5.53994004e-05
 1.10395648e-05 5.87233448e-06 3.13858204e-06 1.54420449e-06
 1.02927386e-06 3.90521513e-07 1.95535151e-07 8.99659634e-08]
